# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import

In [ ]:
! pip install lpips

In [ ]:
# Standard imports
import random
import os
import re
import glob
import time
from pathlib import Path
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from typing import List, Tuple, Dict


# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import lpips

print("All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Data Processing

In [ ]:
# !unzip -q -o "/content/drive/MyDrive/CS230/PlantDS_Merged.zip" -d "/content/drive/MyDrive/CS230"

In [ ]:
def scan_available_plants(data_root):

    path = Path(data_root)
    # find all plantXXXXX_day01.png
    files = list(path.glob("plant*_day*.png"))
    plants = list(path.glob("plant*_day01.png"))

    plant_ids = []
    for f in plants:
        try:
            pid = int(f.name.split('_')[0].replace('plant', ''))
            plant_ids.append(pid)
        except:
            pass

    plant_ids.sort()
    print(f"Found {len(files)} images")
    print(f"Found {len(plant_ids)} plants")
    return plant_ids


class Config:
    DATA_ROOT = "/content/drive/MyDrive/CS230/PlantDS_Merged/Seg"

    # Config
    IMG_SIZE = 128
    BATCH_SIZE = 32
    NUM_WORKERS = 2

    INPUT_DAYS = [1, 2, 3, 4, 5]
    TARGET_DAY = 10

    ALL_PLANTS = scan_available_plants(DATA_ROOT)

    if len(ALL_PLANTS) == 0:
        print(f"WARNING: No Data in {DATA_ROOT}.")
        TRAIN_PLANTS = []
        VAL_PLANTS = []
        TEST_PLANTS = []
    else:
        # Seed
        random.seed(42)
        shuffled = ALL_PLANTS.copy()
        random.shuffle(shuffled)

        # 90% Train, 5% Val, 5% Test
        n_total = len(shuffled)
        n_train = 1850
        n_val = 95

        TRAIN_PLANTS = shuffled[925:n_train]
        VAL_PLANTS = shuffled[n_train : n_train + n_val]
        TEST_PLANTS = shuffled[n_train + n_val:]

        print(f"Data Structure:")
        print(f"   - Total:  {n_total}")
        print(f"   - Train: {len(TRAIN_PLANTS)}")
        print(f"   - Valid: {len(VAL_PLANTS)}")
        print(f"   - Test: {len(TEST_PLANTS)}")


class PlantGrowthDataset(Dataset):
    def __init__(self, data_root, plant_ids, input_days=[1,2,3,4,5],
                 target_day=10, img_size=128):
        self.data_root = Path(data_root)
        self.plant_ids = plant_ids
        self.input_days = input_days
        self.target_day = target_day
        self.img_size = img_size
        self.transform = self._get_transforms()

    def _get_transforms(self):

        common_transforms = [
            transforms.Resize((self.img_size, self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]


        return transforms.Compose(common_transforms)

    def _get_image_path(self, plant_id, day):
        filename = f"plant{plant_id:05d}_day{day:02d}.png"
        return self.data_root / filename

    def __len__(self):
        return len(self.plant_ids)

    def __getitem__(self, idx):
        plant_id = self.plant_ids[idx]

        # Day 1-5
        input_images = []
        for day in self.input_days:
            img_path = self._get_image_path(plant_id, day)
            if not img_path.exists():
                raise FileNotFoundError(f"Missing image: {img_path}")

            img = Image.open(img_path).convert('RGB')
            img_tensor = self.transform(img)
            input_images.append(img_tensor)

        input_images = torch.stack(input_images, dim=0) # [5, 3, H, W]

        # Day 10
        target_path = self._get_image_path(plant_id, self.target_day)
        if not target_path.exists():
             raise FileNotFoundError(f"Missing target: {target_path}")

        target_img = Image.open(target_path).convert('RGB')
        target_image = self.transform(target_img)

        return input_images, target_image, plant_id


def create_dataloaders(config):

    train_dataset = PlantGrowthDataset(
        config.DATA_ROOT, config.TRAIN_PLANTS, config.INPUT_DAYS,
        config.TARGET_DAY, config.IMG_SIZE
    )
    val_dataset = PlantGrowthDataset(
        config.DATA_ROOT, config.VAL_PLANTS, config.INPUT_DAYS,
        config.TARGET_DAY, config.IMG_SIZE
    )
    test_dataset = PlantGrowthDataset(
        config.DATA_ROOT, config.TEST_PLANTS, config.INPUT_DAYS,
        config.TARGET_DAY, config.IMG_SIZE
    )

    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                             shuffle=True, num_workers=config.NUM_WORKERS,
                             pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                           shuffle=False, num_workers=config.NUM_WORKERS,
                           pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                            shuffle=False, num_workers=config.NUM_WORKERS,
                            pin_memory=True)

    return train_loader, val_loader, test_loader

if __name__ == "__main__":

    try:
        config = Config()
        if len(config.TRAIN_PLANTS) > 0:
            train_loader, val_loader, test_loader = create_dataloaders(config)

            inputs, targets, ids = next(iter(train_loader))
            print("\nDataLoader Test Success!")
            print(f"Inputs Shape: {inputs.shape} (Batch, Days, Channels, H, W)")
            print(f"Targets Shape: {targets.shape}")
            print(f"Plant IDs: {ids.tolist()}")


            def denorm(x): return x * 0.5 + 0.5
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 2, 1)
            plt.imshow(denorm(inputs[0, 0]).permute(1, 2, 0).clip(0, 1))
            plt.title(f"Plant {ids[0]}: Day 1 (Input)")
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(denorm(targets[0]).permute(1, 2, 0).clip(0, 1))
            plt.title(f"Plant {ids[0]}: Day 10 (Target)")
            plt.axis('off')
            plt.show()
    except Exception as e:
        print(f"Error: {e}")

data_config = Config()

In [ ]:
# Test Loading

inputs, targets, plant_ids = next(iter(train_loader))

print(f"\nSuccessfully loaded batch")
print(f"   Input shape: {inputs.shape}")
print(f"   Target shape: {targets.shape}")
print(f"   Plant IDs: {plant_ids.tolist()}")

# Training Setup


In [ ]:
#  Configuration

class TrainConfig:
    EPOCHS = 40
    LEARNING_RATE = 5e-6
    WEIGHT_DECAY = 5e-7

    LPIPS_WEIGHT = 1
    L1_WEIGHT = 1.
    PERCEPTUAL_WEIGHT = 0
    PSNR_WEIGHT = 0.00000001
    BACKGROUND_WEIGHT = 2.
    COLOR_WEIGHT = 1.

    USE_MASK = {
        'lpips': False,
        'l1': False,
        'perc': False,
        'psnr': False,
        'bg': True
    }
    MASK_THRESHOLD = -0.9

    USE_SCHEDULER = True
    SCHEDULER_TYPE = 'plateau'
    PATIENCE = 10

    SAVE_DIR = "/content/drive/MyDrive/CS230/PlantDS_1_checkpoints"
    SAVE_EVERY = 5

    VIS_DIR = "/content/drive/MyDrive/CS230/PlantDS_1_visualizations"
    VIS_EVERY = 5
    NUM_VIS_SAMPLES = 4

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    USE_AMP = True

In [ ]:
# Mask Extraction Functions

def compute_mask(image, threshold=-0.9):

    image_mean = image.mean(dim=1, keepdim=True)
    mask = (image_mean > threshold).float()

    return mask

print("Verifying mask computation...")

inputs, targets, _ = next(iter(train_loader))

mask = compute_mask(targets[0:1])

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original
target_vis = ((targets[0] + 1) / 2).permute(1, 2, 0).cpu().numpy()
axes[0].imshow(np.clip(target_vis, 0, 1))
axes[0].set_title('Original Target')
axes[0].axis('off')

# Computed mask
mask_vis = mask[0, 0].cpu().numpy()
axes[1].imshow(mask_vis, cmap='gray')
axes[1].set_title('Computed Mask (White=Plant, Black=Background)')
axes[1].axis('off')

# Masked
target_masked = (targets[0:1] * mask)[0]
target_masked_vis = ((target_masked + 1) / 2).permute(1, 2, 0).cpu().numpy()
axes[2].imshow(np.clip(target_masked_vis, 0, 1))
axes[2].set_title('Masked Target (Only Plant)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Mask coverage: {mask.mean().item()*100:.1f}% of image")
print("Mask computation verified. Losses will only be computed on plant regions.")

print("Checking mask stability...")

for i, (inputs, targets, _) in enumerate(train_loader):
    mask = compute_mask(targets)
    coverage = mask.mean().item()
    mask_sum = mask.sum().item()

    print(f"Batch {i}: Coverage={coverage*100:.1f}%, Mask sum={mask_sum:.0f}")

    if i >= 3:
        break

In [ ]:
# Loss Functions

class LPIPSLoss(nn.Module):

    def __init__(self, net='alex', device='cuda', use_mask=True):

        super().__init__()
        self.use_mask = use_mask
        self.lpips_model = lpips.LPIPS(net=net).to(device)

        # Freeze parameters
        for param in self.lpips_model.parameters():
            param.requires_grad = False
        self.device = device

    def apply_mask_to_image(self, image, mask):

        if mask.shape[1] == 1:
            mask = mask.expand_as(image)

        masked_image = image * mask + (-1.0) * (1 - mask)
        return masked_image

    def forward(self, pred, target, mask=None):

        if self.use_mask and mask is not None:
            pred_input = self.apply_mask_to_image(pred, mask)
            target_input = self.apply_mask_to_image(target, mask)
        else:
            pred_input = pred
            target_input = target

        # LPIPS expects images in [-1, 1] range
        # Compute LPIPS distance
        lpips_distance = self.lpips_model(pred_input, target_input)

        return lpips_distance.mean()


class PerceptualLoss(nn.Module):
# Perceptual loss using VGG16

    def __init__(self, device='cuda', use_mask=TrainConfig.USE_MASK['perc']):
        super().__init__()
        vgg = models.vgg16(weights='DEFAULT').features.eval().to(device)
        for param in vgg.parameters():
            param.requires_grad = False
        self.feature_layers = [3, 8, 15, 22]
        self.vgg = vgg
        self.criterion = nn.L1Loss()
        self.use_mask = use_mask

        self.mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1)

    def normalize_vgg(self, x):
        x = (x + 1) / 2
        return (x - self.mean) / self.std

    def apply_mask_to_image(self, image, mask):
    # Apply mask to image before VGG
        if mask.shape[1] == 1:
            mask = mask.expand_as(image)

        mean_rgb = torch.tensor([0.485, 0.456, 0.406], device=image.device).view(1, 3, 1, 1)
        masked_image = image * mask + mean_rgb * (1 - mask)
        return masked_image

    def forward(self, pred, target, mask=None):
        if self.use_mask and mask is not None:
            pred_input = self.apply_mask_to_image(pred, mask)
            target_input = self.apply_mask_to_image(target, mask)
        else:
            pred_input = pred
            target_input = target

        pred_vgg = self.normalize_vgg(pred_input)
        target_vgg = self.normalize_vgg(target_input)

        loss = 0.0
        for i, layer in enumerate(self.vgg):
            pred_vgg = layer(pred_vgg)
            target_vgg = layer(target_vgg)

            if i in self.feature_layers:
                loss += self.criterion(pred_vgg, target_vgg)

        return loss


class PSNRLoss(nn.Module):

    def __init__(self, max_val=2.0, use_mask=TrainConfig.USE_MASK['psnr']):
        super().__init__()
        self.max_val = max_val
        self.use_mask = use_mask

    def forward(self, pred, target, mask=None):
        if self.use_mask and mask is not None:

            pred_masked = pred * mask
            target_masked = target * mask
            mse = torch.sum((pred_masked - target_masked) ** 2) / (torch.sum(mask) * pred.shape[1] + 1e-10)
        else:
            mse = torch.mean((pred - target) ** 2)

        mse = torch.clamp(mse, min=1e-10)
        psnr = 20 * torch.log10(self.max_val / torch.sqrt(mse))
        return -psnr


class BackgroundRegularizationLoss(nn.Module):
# Penalize non-black predictions in background regions

    def __init__(self, target_value=-1.0):
        super().__init__()
        self.target_value = target_value

    def forward(self, pred, mask):

        background_mask = torch.ones_like(mask, device=pred.device) - mask

        target = torch.full_like(pred, self.target_value, device=pred.device)

        background_error = (pred - target) ** 2

        masked_error = background_error * background_mask

        num_background_pixels = background_mask.sum() * pred.shape[1]

        if num_background_pixels > 0:
            loss = masked_error.sum() / num_background_pixels
        else:
            loss = torch.tensor(0.0, device=pred.device)

        return loss


class ColorDifferenceLoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.criterion = nn.MSELoss(reduction='none')


    def forward(self, pred, target, mask):

        if mask is None:
            # No mask, use full image
            return self.criterion(pred, target).mean()

        # Expand mask to 3 channels
        if mask.shape[1] == 1:
            mask_3ch = mask.expand_as(pred)
        else:
            mask_3ch = mask

        # Compute color difference
        color_diff = self.criterion(pred, target)

        # Apply mask and compute mean over masked pixels
        masked_diff = color_diff * mask_3ch

        # Count masked pixels (3 channels)
        num_masked_pixels = mask_3ch.sum()

        if num_masked_pixels > 0:
            loss = masked_diff.sum() / num_masked_pixels
        else:
            # Fallback if mask is empty
            loss = color_diff.mean()

        return loss


class CombinedLoss(nn.Module):

    def __init__(self, lpips_weight=1., l1_weight=0, perceptual_weight=0.,
                 psnr_weight=0., background_weight=1., color_weight=1.,
                 device='cuda', use_mask={'lpips':False, 'l1':False, 'perc':False, 'psnr':False, 'bg':True}):
        super().__init__()
        self.lpips_weight = lpips_weight
        self.l1_weight = l1_weight
        self.perceptual_weight = perceptual_weight
        self.psnr_weight = psnr_weight
        self.background_weight = background_weight
        self.color_weight = color_weight
        self.use_mask = use_mask

        self.lpips_loss = LPIPSLoss(net='alex', device=device, use_mask=use_mask['lpips']) if lpips_weight > 0 else None
        self.l1_loss = nn.L1Loss(reduction='none') # if l1_weight > 0 else None
        self.perceptual_loss = PerceptualLoss(device, use_mask['perc']) if perceptual_weight > 0 else None
        self.psnr_loss = PSNRLoss(use_mask=use_mask['psnr']) if psnr_weight > 0 else None
        self.background_loss = BackgroundRegularizationLoss() if background_weight > 0 else None
        self.color_loss = ColorDifferenceLoss() if color_weight > 0 else None

    def forward(self, pred, target):
        # Compute mask
        mask = compute_mask(target)
        mask_coverage = mask.sum() / mask.numel()

        if mask_coverage < 0.01:
            mask = None

        # L1 Loss with masking
        l1 = self.l1_loss(pred, target)
        if self.use_mask['l1'] and mask is not None:
            l1 = (l1 * mask).sum() / (mask.sum() * pred.shape[1] + 1e-10)
        else:
            l1 = l1.mean()

        # VGG Perceptual Loss
        if self.perceptual_loss:
            perc = self.perceptual_loss(pred, target, mask)
        else:
            perc = torch.tensor(0.0, device=pred.device)

        # LPIPS Loss

        if self.lpips_loss:
            lpips = self.lpips_loss(pred, target, mask)
        else:
            lpips = torch.tensor(0.0, device=pred.device)

        # PSNR Loss
        if self.psnr_loss:
            psnr = self.psnr_loss(pred, target, mask)
        else:
            psnr = torch.tensor(0.0, device=pred.device)

        # Background Loss
        if self.background_loss and self.use_mask['bg'] and mask is not None:
            bg = self.background_loss(pred, mask)
        else:
            bg = torch.tensor(0.0, device=pred.device)

        # Color Difference Loss
        if self.color_loss and self.use_mask['bg'] and mask is not None:
            color = self.color_loss(pred, target, mask)
        else:
            color = torch.tensor(0.0, device=pred.device)

        # Combined Loss
        total = (self.lpips_weight * lpips +
                self.l1_weight * l1 +
                self.perceptual_weight * perc +
                self.psnr_weight * psnr +
                self.background_weight * bg +
                self.color_weight * color)

        return total, lpips, l1, perc, psnr, bg, color

In [ ]:
# Training Functions

def calculate_psnr(pred, target):
# Calculate Peak Signal-to-Noise Ratio
    mse = torch.mean((pred - target) ** 2)

    if mse < 1e-10:
        return torch.tensor(100.0, device=pred.device)

    psnr = 20 * torch.log10(2.0 / torch.sqrt(mse))
    return psnr


def train_one_epoch(model, loader, criterion, optimizer, scaler, device, epoch):

    model.train()
    total_loss, total_lpips, total_l1, total_psnr, total_bg, total_color = 0, 0, 0, 0, 0, 0
    pbar = tqdm(loader, desc=f"Epoch {epoch} [Train]")

    for inputs, targets, _ in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast('cuda', enabled=scaler is not None):
            outputs = model(inputs)
            loss, lpips_loss, l1_loss, perc_loss, psnr_loss, bg_loss, color_loss = criterion(outputs, targets)

        if scaler:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        total_loss += loss.item()
        total_lpips += lpips_loss.item()
        total_l1 += l1_loss.item()
        total_psnr += psnr_loss.item()
        total_bg += bg_loss.item()
        total_color += color_loss.item()

        pbar.set_postfix({
            'total loss': f'{loss.item():.4f}',
            'lpips': f'{lpips_loss.item():.4f}',
            'l1': f'{l1_loss.item():.4f}',
            'perc': f'{perc_loss.item():.4f}',
            'psnr': f'{-psnr_loss.item():.4f}',
            'bg': f'{bg_loss.item():.4f}',
            'color': f'{color_loss.item():.4f}'
            })

    return (total_loss/len(loader), total_lpips/len(loader), total_l1/len(loader),
            total_psnr/len(loader), total_bg/len(loader), total_color/len(loader))

def validate(model, loader, criterion, device, epoch):

    model.eval()
    total_loss, total_lpips, total_l1, total_psnr, total_bg, total_color = 0, 0, 0, 0, 0, 0

    with torch.no_grad():
        for inputs, targets, _ in tqdm(loader, desc=f"Epoch {epoch} [Val]"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss, lpips_loss, l1_loss, perc_loss, psnr_loss, bg_loss, color_loss = criterion(outputs, targets)
            psnr = calculate_psnr(outputs, targets)

            total_loss += loss.item()
            total_lpips += lpips_loss.item()
            total_l1 += l1_loss.item()
            total_psnr += psnr_loss.item()
            total_bg += bg_loss.item()
            total_color += color_loss.item()

    return (total_loss/len(loader), total_lpips/len(loader), total_l1/len(loader),
            total_psnr/len(loader), total_bg/len(loader), total_color/len(loader))


def save_checkpoint(model, optimizer, scheduler, epoch, history, config, is_best=False):

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'history': history,
    }
    save_dir = Path(config.SAVE_DIR)
    save_dir.mkdir(parents=True, exist_ok=True)

    torch.save(checkpoint, save_dir / 'checkpoint_latest.pth')
    if is_best:
        torch.save(checkpoint, save_dir / 'checkpoint_best.pth')
        print(f"Best checkpoint saved!")
    if epoch % config.SAVE_EVERY == 0:
        torch.save(checkpoint, save_dir / f'checkpoint_epoch_{epoch:03d}.pth')

print("Training functions defined!")

In [ ]:
# Visualization Function

def denormalize(tensor):
    return (tensor + 1) / 2


def visualize_predictions(model, loader, device, epoch, save_dir, num_samples=4):
    model.eval()
    inputs, targets, plant_ids = next(iter(loader))
    inputs, targets = inputs.to(device), targets.to(device)

    with torch.no_grad():
        predictions = model(inputs)

    # Randomly select samples from the batch
    batch_size = inputs.shape[0]
    num_samples = min(num_samples, batch_size)
    random_indices = torch.randperm(batch_size)[:num_samples]

    fig, axes = plt.subplots(num_samples, 7, figsize=(21, 3*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)

    for idx, i in enumerate(random_indices):
        for j in range(5):
            img = denormalize(inputs[i,j]).permute(1,2,0).cpu().numpy()
            axes[idx,j].imshow(np.clip(img, 0, 1))
            axes[idx,j].set_title(f'Day {j+1}')
            axes[idx,j].axis('off')

        pred = denormalize(predictions[i]).permute(1,2,0).cpu().numpy()
        axes[idx,5].imshow(np.clip(pred, 0, 1))
        axes[idx,5].set_title('Predicted', color='blue')
        axes[idx,5].axis('off')

        target = denormalize(targets[i]).permute(1,2,0).cpu().numpy()
        axes[idx,6].imshow(np.clip(target, 0, 1))
        axes[idx,6].set_title('Ground Truth', color='green')
        axes[idx,6].axis('off')

    plt.suptitle(f'Epoch {epoch}', fontsize=14, fontweight='bold')
    plt.tight_layout()

    save_path = Path(save_dir) / f'predictions_epoch_{epoch:03d}.png'
    save_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

print("Visualization functions defined!")

In [ ]:
def train_model(model, train_loader, val_loader, config):

    device = config.DEVICE
    model = model.to(device)

    criterion = CombinedLoss(
        config.LPIPS_WEIGHT,
        config.L1_WEIGHT,
        config.PERCEPTUAL_WEIGHT,
        config.PSNR_WEIGHT,
        config.BACKGROUND_WEIGHT,
        config.COLOR_WEIGHT,
        device,
        use_mask=config.USE_MASK
    )

    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=config.PATIENCE)
    scaler = torch.cuda.amp.GradScaler() if config.USE_AMP else None

    history = {
        'train_loss': [], 'val_loss': [],
        'train_lpips': [], 'train_l1': [], 'train_psnr': [], 'train_bg': [], 'train_color': [],
        'val_lpips': [], 'val_l1': [], 'val_psnr': [], 'val_bg': [], 'val_color': [],
        'lr': []
    }
    best_val_loss = float('inf')

    print(f"\nTraining on {device}")
    print(f"Epochs: {config.EPOCHS} | LR: {config.LEARNING_RATE}")

    for epoch in range(1, config.EPOCHS + 1):
        print(f"\n{'='*60}\nEpoch {epoch}/{config.EPOCHS}")

        train_loss, train_lpips, train_l1, train_psnr, train_bg, train_color = train_one_epoch(
            model, train_loader, criterion, optimizer, scaler, device, epoch
        )
        val_loss, val_lpips, val_l1, val_psnr, val_bg, val_color = validate(
            model, val_loader, criterion, device, epoch
        )

        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        new_lr = optimizer.param_groups[0]['lr']
        if old_lr != new_lr:
            print(f"Learning rate reduced: {old_lr:.2e} -> {new_lr:.2e}")

        history['train_loss'].append(train_loss)
        history['train_lpips'].append(train_lpips)
        history['train_l1'].append(train_l1)
        history['train_psnr'].append(-train_psnr)
        history['train_bg'].append(train_bg)
        history['train_color'].append(train_color)

        history['val_loss'].append(val_loss)
        history['val_lpips'].append(val_lpips)
        history['val_l1'].append(val_l1)
        history['val_psnr'].append(val_psnr)
        history['val_bg'].append(val_bg)
        history['val_color'].append(val_color)

        history['lr'].append(optimizer.param_groups[0]['lr'])

        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"Train PSNR: {train_psnr:.2f}dB |Val PSNR: {val_psnr:.2f}dB ")

        is_best = val_loss < best_val_loss
        if is_best:
            best_val_loss = val_loss

        save_checkpoint(model, optimizer, scheduler, epoch, history, config, is_best)

        if epoch % config.VIS_EVERY == 0:
            visualize_predictions(model, val_loader, device, epoch,
                                config.VIS_DIR, config.NUM_VIS_SAMPLES)

    print(f"\nTraining complete! Best val loss: {best_val_loss:.4f}")

    return model, history

In [ ]:
def plot_training_history(history, save_path=None):
# Plot training and validation losses over epochs including LPIPS

    import matplotlib.pyplot as plt
    import numpy as np

    epochs = range(1, len(history['train_loss']) + 1)

    fig, axes = plt.subplots(2, 3, figsize=(20, 10))

    # Plot 1: Total Loss
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Total Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: PSNR
    valid_inverted = [-x for x in history['val_psnr']]
    axes[0, 1].plot(epochs, valid_inverted, label='Valid', linewidth=2)
    axes[0, 1].plot(epochs, history['train_psnr'], label='Train', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('PSNR (dB)')
    axes[0, 1].set_title('PSNR')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: L1
    axes[0, 2].plot(epochs, history['val_l1'], label='Valid', linewidth=2)
    axes[0, 2].plot(epochs, history['train_l1'], label='Train', linewidth=2)
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('L1')
    axes[0, 2].set_title('L1')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    # Plot 4: Individual Train Loss Components
    axes[1, 1].plot(epochs, history['train_l1'], label='L1 Loss', linewidth=2)
    axes[1, 1].plot(epochs, history['train_color'], label='Color Loss', linewidth=2)
    axes[1, 1].plot(epochs, history['train_bg'], label='Background Loss', linewidth=2)
    axes[1, 1].plot(epochs, history['train_lpips'], label='LPIPS Loss', linewidth=2)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].set_title('Training Loss Components')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    # Plot 5: Individual Valid Loss Components
    axes[1, 0].plot(epochs, history['val_l1'], label='L1 Loss', linewidth=2)
    axes[1, 0].plot(epochs, history['val_color'], label='Color Loss', linewidth=2)
    axes[1, 0].plot(epochs, history['val_bg'], label='Background Loss', linewidth=2)
    axes[1, 0].plot(epochs, history['val_lpips'], label='LPIPS Loss', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].set_title('Validation Loss Components')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 6: Learning Rate
    axes[1, 2].plot(epochs, history['lr'], 'purple', linewidth=2)
    axes[1, 2].set_xlabel('Epoch')
    axes[1, 2].set_ylabel('Learning Rate')
    axes[1, 2].set_title('Learning Rate Schedule')
    axes[1, 2].set_yscale('log')
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Plot saved to {save_path}")

    plt.show()

    # Print summary statistics
    print("\nTraining Summary:")
    print("="*60)
    print(f"Initial Train Loss: {history['train_loss'][0]:.4f}")
    print(f"Final Train Loss: {history['train_loss'][-1]:.4f}")
    print(f"Best Train Loss: {min(history['train_loss']):.4f} (Epoch {np.argmin(history['train_loss'])+1})")
    print()
    print(f"Initial Val Loss: {history['val_loss'][0]:.4f}")
    print(f"Final Val Loss: {history['val_loss'][-1]:.4f}")
    print(f"Best Val Loss: {min(history['val_loss']):.4f} (Epoch {np.argmin(history['val_loss'])+1})")

    if 'val_psnr' in history:
        print()
        print(f"Initial Val PSNR: {history['val_psnr'][0]:.2f} dB")
        print(f"Final Val PSNR: {history['val_psnr'][-1]:.2f} dB")
        print(f"Best Val PSNR: {max(history['val_psnr']):.2f} dB (Epoch {np.argmax(history['val_psnr'])+1})")

    if 'val_lpips' in history:  # NEW
        print()
        print(f"Initial Val LPIPS: {history['val_lpips'][0]:.4f}")
        print(f"Final Val LPIPS: {history['val_lpips'][-1]:.4f}")
        print(f"Best Val LPIPS: {min(history['val_lpips']):.4f} (Epoch {np.argmin(history['val_lpips'])+1})")

    print("="*60)

# 2D U-Net Architecture

In [ ]:
class DoubleConv(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels//2, 2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX//2, diffX-diffX//2, diffY//2, diffY-diffY//2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class UNet(nn.Module):
# U-Net for temporal plant growth prediction
    def __init__(self, n_channels=15, n_classes=3, bilinear=True):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // (2 if bilinear else 1))

        self.up1 = Up(1024, 512 // (2 if bilinear else 1), bilinear)
        self.up2 = Up(512, 256 // (2 if bilinear else 1), bilinear)
        self.up3 = Up(256, 128 // (2 if bilinear else 1), bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = nn.Conv2d(64, n_classes, 1)
        self.output_activation = nn.Tanh()

    def forward(self, x):
        # Reshape: [B, 5, 3, H, W] -> [B, 15, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B, T * C, H, W)

        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # Decoder
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        # Output
        logits = self.outc(x)
        return self.output_activation(logits)

    def count_parameters(self):
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total, trainable

print("U-Net model defined!")

In [ ]:
# Create configurations
train_config = TrainConfig()

# Create dataloaders
print("\n Loading data...")
train_loader, val_loader, test_loader = create_dataloaders(data_config)

# Create model
print("\n Creating model...")
model = UNet(n_channels=15, n_classes=3, bilinear=True)
total_params, _ = model.count_parameters()
print(f"Model parameters: {total_params:,}")

# Try one batch
inputs, targets, ids = next(iter(train_loader))
print(f"\nBatch shapes: {inputs.shape} -> {targets.shape}")

# Start Training
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)

trained_model, history = train_model(model, train_loader, val_loader, train_config)

print("ALL DONE!")

# 3D U-Net Architecture

In [ ]:
class DoubleConv3D(nn.Module):

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels

        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down3D(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        # Only downsample spatially (H, W), keep temporal dimension
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2)),  # (T, H, W)
            DoubleConv3D(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up3D(nn.Module):

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            # Use trilinear interpolation for 3D
            self.up = nn.Upsample(scale_factor=(1, 2, 2), mode='trilinear', align_corners=True)
            self.conv = DoubleConv3D(in_channels, out_channels, in_channels // 2)
        else:
            # Transposed convolution - only upsample spatially
            self.up = nn.ConvTranspose3d(
                in_channels, in_channels // 2,
                kernel_size=(1, 2, 2),
                stride=(1, 2, 2)
            )
            self.conv = DoubleConv3D(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # Handle size mismatch (T, H, W)
        diffT = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        diffX = x2.size()[4] - x1.size()[4]

        x1 = F.pad(x1, [
            diffX // 2, diffX - diffX // 2,
            diffY // 2, diffY - diffY // 2,
            diffT // 2, diffT - diffT // 2
        ])

        # Concatenate along channel dimension
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class UNet3D(nn.Module):

    def __init__(self, n_frames=5, input_channels=3, output_channels=3, bilinear=True):
        super().__init__()
        self.n_frames = n_frames
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.bilinear = bilinear

        self.inc = DoubleConv3D(input_channels, 64)

        # Encoder (downsampling spatially)
        self.down1 = Down3D(64, 128)
        self.down2 = Down3D(128, 256)
        self.down3 = Down3D(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down3D(512, 1024 // factor)

        # Decoder (upsampling spatially)
        self.up1 = Up3D(1024, 512 // factor, bilinear)
        self.up2 = Up3D(512, 256 // factor, bilinear)
        self.up3 = Up3D(256, 128 // factor, bilinear)
        self.up4 = Up3D(128, 64, bilinear)

        # Final output layer
        self.outc = nn.Conv3d(64, output_channels, kernel_size=(n_frames, 1, 1))

        # Output activation
        self.output_activation = nn.Tanh()

    def forward(self, x):

        B, T, C, H, W = x.shape
        x = x.permute(0, 2, 1, 3, 4)  # [B, 3, 5, H, W]

        # Encoder with skip connections
        x1 = self.inc(x)      # [B, 64, 5, H, W]
        x2 = self.down1(x1)   # [B, 128, 5, H/2, W/2]
        x3 = self.down2(x2)   # [B, 256, 5, H/4, W/4]
        x4 = self.down3(x3)   # [B, 512, 5, H/8, W/8]
        x5 = self.down4(x4)   # [B, 512, 5, H/16, W/16]

        # Decoder with skip connections
        x = self.up1(x5, x4)  # [B, 512, 5, H/8, W/8]
        x = self.up2(x, x3)   # [B, 256, 5, H/4, W/4]
        x = self.up3(x, x2)   # [B, 128, 5, H/2, W/2]
        x = self.up4(x, x1)   # [B, 64, 5, H, W]

        # Final convolution to reduce temporal dimension
        logits = self.outc(x)  # [B, 3, 1, H, W]

        # Remove temporal dimension
        logits = logits.squeeze(2)  # [B, 3, H, W]

        # Activation
        output = self.output_activation(logits)

        return output

    def count_parameters(self):
        """Count total and trainable parameters"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total_params, trainable_params


def test_unet3d():
    """Test the 3D U-Net model"""
    print("=" * 80)
    print("Testing 3D U-Net Model")
    print("=" * 80)

    model = UNet3D(n_frames=5, input_channels=3, output_channels=3, bilinear=True)

    total_params, trainable_params = model.count_parameters()
    print(f" Model Parameters:")
    print(f"   - Total: {total_params:,}")
    print(f"   - Trainable: {trainable_params:,}")
    print(f"   - Size: ~{total_params * 4 / 1024 / 1024:.2f} MB")

    batch_size = 2
    input_tensor = torch.randn(batch_size, 5, 3, 256, 256)

    print(f"Testing forward pass...")
    print(f"   - Input shape: {input_tensor.shape}")

    with torch.no_grad():
        output = model(input_tensor)

    print(f"   - Output shape: {output.shape}")
    print(f"   - Output range: [{output.min():.3f}, {output.max():.3f}]")

    assert output.shape == (batch_size, 3, 256, 256), "Output shape mismatch!"

    print(f"3D U-Net test passed!")
    print("=" * 80)

    return model


print("3D U-Net model defined!")

In [ ]:

train_config = TrainConfig()

# Create dataloaders
print("Loading data...")
train_loader, val_loader, test_loader = create_dataloaders(data_config)

# Create 3D U-Net model
print("Creating 3D U-Net model...")
model = UNet3D(n_frames=5, input_channels=3, output_channels=3, bilinear=True)
total_params, _ = model.count_parameters()
print(f"Model parameters: {total_params:,}")

# Try one batch
inputs, targets, ids = next(iter(train_loader))
print(f"\nBatch shapes: {inputs.shape} -> {targets.shape}")

# Start
print("\n" + "="*80)
print("STARTING TRAINING WITH 3D U-NET")
print("="*80)

trained_model, history = train_model(model, train_loader, val_loader, train_config)

print("ALL DONE!")

# ConvLSTM U-Net Model Architecture

In [ ]:
class DoubleConv(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, 2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # Handle size mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2

        self.conv = nn.Conv2d(
            input_dim + hidden_dim,
            4 * hidden_dim,  # Gates: input, forget, output, cell
            kernel_size,
            padding=padding
        )

    def forward(self, x, hidden_state):
        h_cur, c_cur = hidden_state
        combined = torch.cat([x, h_cur], dim=1)
        gates = self.conv(combined)

        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)
        i = torch.tanh(i)
        f = torch.tanh(f)
        o = torch.tanh(o)
        g = torch.tanh(g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, (h_next, c_next)


class ConvLSTMUNet(nn.Module):

    def __init__(self):
        super().__init__()

        # Encoder
        self.enc1 = DoubleConv(3, 64)
        self.enc2 = Down(64, 128)
        self.enc3 = Down(128, 256)

        # ConvLSTM
        self.convlstm = ConvLSTMCell(256, 256, kernel_size=3)

        # Decoder
        self.up1 = Up(256 + 256, 128, bilinear=True)  # 256 from ConvLSTM + 256 from skip
        self.up2 = Up(128 + 128, 64, bilinear=True)  # 128 from up1 + 128 from skip
        self.up3 = Up(64 + 64, 64, bilinear=True)   # 64 from up2 + 64 from skip
        self.out = nn.Conv2d(64, 3, 1)
        self.Tanh = nn.Tanh()

    def forward(self, x):
        # x: [B, T=5, C=3, H, W]
        B, T, C, H, W = x.shape

        # Encode each frame and store all features for skip connections
        all_f1, all_f2, all_f3 = [], [], []

        for t in range(T):
            frame = x[:, t]          # [B, 3, H, W]
            f1 = self.enc1(frame)       # [B, 64, H, W]
            f2 = self.enc2(f1)         # [B, 128, H/2, W/2]
            f3 = self.enc3(f2)         # [B, 256, H/4, W/4]
            all_f1.append(f1)
            all_f2.append(f2)
            all_f3.append(f3)

        # Process temporal sequence with ConvLSTM
        h_size = (B, 256, H//4, W//4)
        h = torch.zeros(h_size, device=x.device)
        c = torch.zeros(h_size, device=x.device)

        for t in range(T):
            h, (h, c) = self.convlstm(all_f3[t], (h, c))

        out = self.up1(h, all_f3[-1])     # [B, 128, H/2, W/2]
        out = self.up2(out, all_f2[-1])    # [B, 64, H, W]
        out = self.up3(out, all_f1[-1])    # [B, 64, H, W]
        out = self.out(out)           # [B, 3, H, W]

        return self.Tanh(out)

    def count_parameters(self):
        """Count total and trainable parameters"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total_params, trainable_params


print("ConvLSTM U-Net model defined!")

In [ ]:
train_config = TrainConfig()

# Create dataloaders
print("Loading data...")
train_loader, val_loader, test_loader = create_dataloaders(data_config)

# Create ConvLSTM U-Net model
print("Creating ConvLSTM U-Net model...")
model = ConvLSTMUNet()
total_params, _ = model.count_parameters()
print(f"Model parameters: {total_params:,}")

# Try one batch
inputs, targets, ids = next(iter(train_loader))
print(f"\nBatch shapes: {inputs.shape} -> {targets.shape}")

# Start
print("\n" + "="*80)
print("STARTING TRAINING WITH CONVLSTM U-NET")
print("="*80)

trained_model, history = train_model(model, train_loader, val_loader, train_config)

print("ALL DONE!")

# End of Training (Result)

In [ ]:
plot_training_history(history, save_path=f"{train_config.VIS_DIR}/training_curves.png")

import lpips
# Load best checkpoint
checkpoint = torch.load(f"{train_config.SAVE_DIR}/checkpoint_best.pth")
trained_model.load_state_dict(checkpoint['model_state_dict'])

# Evaluate on test set
print("\n Evaluating on test set...")
device = train_config.DEVICE
trained_model.eval()

test_loss, test_psnr = 0, 0
criterion = CombinedLoss(1.0, 1.0, 0.0, 0.002, 2.0, 1.0, device)

with torch.no_grad():
    for inputs, targets, _ in tqdm(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = trained_model(inputs)
        loss, lpips, l1, perc, psnr, bg, color = criterion(outputs, targets)
        psnr = calculate_psnr(outputs, targets)
        test_loss += loss.item()
        test_psnr += psnr.item()

avg_test_loss = test_loss / len(test_loader)
avg_test_psnr = test_psnr / len(test_loader)

print(f"\n Test Results:")
print(f"   Loss: {avg_test_loss:.4f}")
print(f"   PSNR: {avg_test_psnr:.2f} dB")

def predict_single_plant(model, plant_id, data_config, device):
    """Predict growth for a single plant"""
    dataset = PlantGrowthDataset(
        data_config.DATA_ROOT, [plant_id],
        data_config.INPUT_DAYS, data_config.TARGET_DAY,
        data_config.IMG_SIZE
    )

    inputs, target, _ = dataset[0]
    inputs = inputs.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        prediction = model(inputs)

    fig, axes = plt.subplots(1, 7, figsize=(21, 3))
    for i in range(5):
        img = denormalize(inputs[0,i]).permute(1,2,0).cpu().numpy()
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f'Day {i+1}')
        axes[i].axis('off')

    pred = denormalize(prediction[0]).permute(1,2,0).cpu().numpy()
    axes[5].imshow(np.clip(pred, 0, 1))
    axes[5].set_title('Predicted Day 10', color='blue', fontweight='bold')
    axes[5].axis('off')

    gt = denormalize(target).permute(1,2,0).cpu().numpy()
    axes[6].imshow(np.clip(gt, 0, 1))
    axes[6].set_title('Ground Truth', color='green', fontweight='bold')
    axes[6].axis('off')

    plt.suptitle(f'Plant {plant_id:02d} Growth Prediction', fontsize=14)
    plt.tight_layout()
    plt.show()

# Predict for plant 16, 21, 26, 27 ,34 ,5
for i in [16, 21, 26, 27 ,34 ,5]:
  predict_single_plant(trained_model, plant_id=i, data_config=data_config, device=train_config.DEVICE)